In [1]:
from datetime import datetime
from datetime import date, timedelta
import pandas as pd
from tsc.text_stream_clustering import TextStreamClustering

## Подготовливаем документы для кластеризации

In [2]:
df = pd.read_csv('../data/april_final_dataset.tsv', sep='\t')
df = df[df.predicts_test==0]
df = df[df.text.notna()]
df.head()

,id,text,Content,heading_id,heading__name,parent,mood,messages_themes__theme__name,messages_themes__theme_id,source_id,...,NER,lemmas_pos_no_stop,revised_cutted,revised_cutted_stopwords,revised_cutted_stopwords_unigrams_only,PostSubscribersCount,AuthorSubscribersCount,cutted_stopwords_unigrams_only,phraser_trigrams,cutted_trigrams
3,447301,"Сейчас бы в море, а не вот это вот все 🥶☔️ дэ ?","Сейчас бы в море, а не вот это вот все 🥶☔️ дэ ?",,,,na,Не определено (апр-май),2551.0,116,...,NaN,море это весь дэ,море,море,море,16215939,16215939,море,NaN,море
5,447302,Чудесный вечер на Премии с Митей Фоминым 🌺 и с...,Чудесный вечер на Премии с Митей Фоминым @mf_a...,,,,na,Не определено (апр-май),2551.0,116,...,NaN,чудесный вечер премия митя::фомин mfagent миша...,вечер премия фомин миша гребенщиков беспокойс...,вечер премия фомин миша гребенщиков беспокойст...,вечер премия фомин миша гребенщиков беспокойст...,1857295,1857295,вечер премия фомин миша гребенщиков беспокойст...,NaN,вечер премия фомин миша гребенщиков беспокойст...
16,449731,Дорогие друзья! Сегодня мы отмечаем День весны...,Дорогие друзья! Сегодня мы отмечаем День весны...,,,,na,!республика кадыров чр регион чеченский_респуб...,2135.0,116,...,рамзан ахмат кадыров,дорогой друг сегодня отмечать день весна труд ...,друг весна труд праздник кадыров житель регио...,весна труд житель регион пример трудолюбие вер...,весна труд праздник житель регион пример трудо...,318982,318982,весна труд праздник житель регион пример трудо...,ахмат-хаджи_азот_цуьнан йойло_глава_чеченский_...,весна труд праздник житель регион пример трудо...
21,444630,Я подписан на два онлайн кинотеатра и даже так...,Я подписан на два онлайн кинотеатра и даже так...,,,,na,Не определено (апр-май),2551.0,114,...,NaN,подписывать онлайн кинотеатр т$тчк$е часто нах...,онлайн кинотеатр фильм вопрос vpn рутрекер,онлайн кинотеатр фильм vpn рутрекер,онлайн кинотеатр фильм vpn рутрекер,390840,390840,онлайн кинотеатр фильм vpn рутрекер,NaN,онлайн кинотеатр фильм vpn рутрекер
27,447290,1 мая! Ура!!! С праздником !!!,1 мая! Ура!!! С праздником !!! #первомай #труд...,,,,na,! труд выходной май праздник первомай длинный_...,2020.0,116,...,NaN,<num> май ур праздник thepangaia,май праздник первомай труд мир май весна,май первомай труд май весна,май праздник первомай труд май весна,1445386,1445386,май праздник первомай труд май весна,NaN,май праздник первомай труд май весна


In [3]:
df['created_at'] =  pd.to_datetime(df['created_at'], format='%Y-%m-%d %H:%M:%S')

In [4]:
docs = []
for idx, text, date in zip(df.id.to_list(), df.text.to_list(), df.created_at.to_list()):
    docs.append({
        'idx': idx,
        'text': text,
        'date': date
    })
len(docs)

15838

## Инициализируем кластеризатор

In [5]:
tsc = TextStreamClustering(device='cuda')

## Добавим пачку документов и произведём первоначальную кластеризацию

In [6]:
# Добавляем документы
tsc.load_docs(docs=docs[:7000])

INFO:TSC:Загружаем и обрабатываем документы
Предобработка документов: 100%|██████████| 7000/7000 [01:18<00:00, 89.04it/s] 
INFO:TSC:Загружено 7000 документов.
INFO:TSC:Создаём эмбеддинги текстов


Batches:   0%|          | 0/219 [00:00<?, ?it/s]

/home/paperspace/anaconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


In [7]:
# производим кластеризацию добавленных документов
tsc.clusterize()

INFO:TSC:Начинаем кластеризацию
INFO:TSC:Кластеризация успешно завершена


In [8]:
# Список кластеров со списком идентификаторов документов
clusters = tsc.get_clusters()

In [9]:
len(clusters)

785

In [10]:
cid = 0
print(len(clusters[cid]))
for i, idx in enumerate(clusters[cid]):
    print(df[df.id == idx].text.to_list())
    if i == 10:
        break

37
['Число пациентов, снятых с медицинского наблюдения, вновь увеличилось. За последние сутки выписано ещё 111 человек. В целом за весь период от COVID-19 на Ставрополье излечились более 48,8 тысячи пациентов. ⠀ Продолжается вакцинация. На сегодня прививку сделали почти 148 тысяч обратившихся. Около 60 процентов из них получили уже оба компонента вакцины. ⠀ Число исследований на коронавирус также растёт и превысило 1 миллион 615 тысяч. Тем, у кого диагностирована инфекция, оказывается медицинская помощь. ⠀ Всё необходимое для этого в крае есть, но медицина не всесильна']
['Вот информация о том, как изменилась санитарно-эпидемиологическая ситуация в нашем крае за прошедшие сутки. ⠀ Число тех, кто сумел преодолеть коронавирус, вновь увеличилось и уже превысило 49 тысяч человек. В том числе, за минувший день снято с медицинского наблюдения 139 пациентов. ⠀ Работа по выявлению новых случаев COVID-19 продолжается. Всего на Ставрополье проведено более 1 миллиона 628 тысяч исследований на кор

## Попробуем выделить субкластеры из большого кластера

In [11]:
subclusters = tsc.get_subcluster(doc_ids=clusters[cid])

In [12]:
len(subclusters)

7

In [14]:
for sc in subclusters:
    print('=====CLUSTER=====')
    for i, idx in enumerate(sc):
        print(df[df.id == idx].text.to_list())
        if i == 2:
            break

=====CLUSTER=====
['Дорогие земляки! Вот информация о том, как изменилась санитарно-эпидемиологическая ситуация в нашем крае за прошедшие сутки. ⠀ Число тех, кто сумел преодолеть коронавирус, вновь увеличилось и уже превысило 49,6 тысячи человек. В том числе, за минувший день выписано и снято с медицинского наблюдения 90 пациентов. ⠀ Работа по выявлению новых случаев COVID-19 продолжается. Охват тестированием растёт. Всего на Ставрополье проведено свыше 1 миллиона 670 тысяч исследований на коронавирус. Заболевшим оказывается медицинская помощь. ⠀ Прививочная кампания также не снижает темпов. Всего прививку от коронавируса уже сделали сввше 181 тысяч человек. В том числе, 103 тысячи из них получили оба компонента вакцины. ⠀ Но инфекция пока не ушла. К сожалению, 4 человека с подтверждённым диагнозом скончались, хотя врачи сделали всё для их спасения. Глубокие соболезнования родным… ⠀ Помните о профилактике. Берегите себя и близких! ⠀ Всем желаю здоровья! ⠀']
['Дорогие земляки! Вот инфор

## Добавим ещё раз документы и произведём кластеризацию

Новые документы добавятся к существующим кластерам, либо из них будут созданы новые кластера.

In [15]:
tsc.load_docs(docs=docs[7001:9000])

INFO:TSC:Загружаем и обрабатываем документы
Предобработка документов: 100%|██████████| 1999/1999 [00:21<00:00, 92.58it/s] 
INFO:TSC:Загружено 8999 документов.
INFO:TSC:Создаём эмбеддинги текстов


Batches:   0%|          | 0/282 [00:00<?, ?it/s]

/home/paperspace/anaconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


In [16]:
tsc.clusterize()
clusters = tsc.get_clusters()
len(clusters)

INFO:TSC:Начинаем кластеризацию
INFO:TSC:Кластеризация успешно завершена


861

In [18]:
# Как видим, документов в этом кластере стало больше - за счёт добавления новых
cid = 0
print(len(clusters[cid]))
for i, idx in enumerate(clusters[cid]):
    print(df[df.id == idx].text.to_list())
    if i == 10:
        break

41
['Число пациентов, снятых с медицинского наблюдения, вновь увеличилось. За последние сутки выписано ещё 111 человек. В целом за весь период от COVID-19 на Ставрополье излечились более 48,8 тысячи пациентов. ⠀ Продолжается вакцинация. На сегодня прививку сделали почти 148 тысяч обратившихся. Около 60 процентов из них получили уже оба компонента вакцины. ⠀ Число исследований на коронавирус также растёт и превысило 1 миллион 615 тысяч. Тем, у кого диагностирована инфекция, оказывается медицинская помощь. ⠀ Всё необходимое для этого в крае есть, но медицина не всесильна']
['Вот информация о том, как изменилась санитарно-эпидемиологическая ситуация в нашем крае за прошедшие сутки. ⠀ Число тех, кто сумел преодолеть коронавирус, вновь увеличилось и уже превысило 49 тысяч человек. В том числе, за минувший день снято с медицинского наблюдения 139 пациентов. ⠀ Работа по выявлению новых случаев COVID-19 продолжается. Всего на Ставрополье проведено более 1 миллиона 628 тысяч исследований на кор

## Получим расширенные данные по каждому кластеру

In [19]:
clusters = tsc.get_clusters_extended()

In [21]:
# идентификаторы документов
print(clusters[0].text_ids)

[455685, 455679, 455689, 455684, 455678, 459707, 14583, 448240, 238993, 23096, 240467, 231611, 238301, 240198, 47915, 239738, 8738, 454938, 441710, 296103, 237133, 458867, 431295, 449090, 530350, 240192, 459702, 32031, 7678, 433903, 454958, 8628, 442267, 439762, 238307, 459703, 26118, 10919, 208242, 207010, 204469]


In [22]:
# Топ-токены для формирования условного заголовка кластера
clusters[0].get_name_by_doc_tokens()

['тысяча', 'человек', 'коронавирус', 'вакцинация', 'число']

In [23]:
# Начало жизни кластера - самы старый документ
clusters[0].get_start_date()

Timestamp('2021-04-16 10:21:03.958975+0000', tz='UTC')

In [24]:
# окончание жизни кластера - самый новый документ
clusters[0].get_end_date()

Timestamp('2021-05-17 15:42:56.778058+0000', tz='UTC')

## Сохраним кластера, чтобы потом можно было бы их использовать ещё раз

In [25]:
tsc.save_clusters(filepath='../temp_data/clusters.clusters')

INFO:TSC:861 кластеров успешно сохранено: `../temp_data/clusters.clusters`


## Загрузим кластера, добавим новые документы

In [28]:
date_from = clusters[0].get_end_date() - timedelta(days=15)
date_from

Timestamp('2021-05-02 15:42:56.778058+0000', tz='UTC')

In [30]:
tsc = TextStreamClustering(device='cuda')
tsc.load_clusters(filepath='../temp_data/clusters.clusters', date_from=date_from)
tsc.load_docs(docs=docs[9001:])
tsc.clusterize()

INFO:TSC:Инициализация
INFO:TSC:Загружено 563 кластеров.
INFO:TSC:Загружаем и обрабатываем документы
Предобработка документов: 100%|██████████| 6837/6837 [01:29<00:00, 76.19it/s] 
INFO:TSC:Загружено 6837 документов.
INFO:TSC:Создаём эмбеддинги текстов


Batches:   0%|          | 0/214 [00:00<?, ?it/s]

/home/paperspace/anaconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")
INFO:TSC:Начинаем кластеризацию
INFO:TSC:Кластеризация успешно завершена


In [31]:
clusters = tsc.get_clusters_extended()

In [32]:
# Это новые тексты, попавшие в данный кластер.
clusters[0].text_ids

[204160, 455691, 203223, 460234, 202308, 582256, 201799, 201366, 21136]

In [33]:
clusters[0].get_name_by_doc_tokens()

['тысяча', 'человек', 'коронавирус', 'число', 'вакцинация']

In [34]:
for idx in clusters[0].text_ids:
    print(df[df.id == idx].text.to_list())

['Дорогие земляки! Начинаем неделю с оперативной информации о ситуации с коронавирусом. ⠀ За прошедшие сутки еще 35 пациентовв нашем крае выписаны из-под меднаблюдения в связи с выздоровлением. В целом, за весь период пандемии число тех, кто смог победить болезнь составляетуже 47 953 человек. ⠀ Продолжается тестированиедля выявления новых случаев передачи инфекции. К сегодняшнему утру проведено 1 миллион 552 тысячи таких исследований. С заболевшими работают врачи. ⠀ Однако победить коронавирус не всегда легко. Несмотря на все усилия, вчера скончались трое пациентов, с подтвержденным COVID-19. Соболезную их родным... ⠀ Защиту от инфекции помогает сформировать вакцинация. Но и об обычныхмерах эпидпрофилактики также не стоит забывать. ⠀ Берегите себя! Всем желаю здоровья! ⠀']
['Вот информация о санитарно-эпидемиологической обстановке в крае по итогам минувшего дня. ⠀ Число выздоровевших от коронавируса увеличилось еще на 47 человек и в целом составило почти 48 тысяч. ⠀ Ежедневно осуществл